In [ ]:
#the idea here is to get practice working with JSON files and generating data etc. from them.

To process an entire paper, all I need to do is use the following import:

from functions_and_classes.paper_processing import process_and_score_paper as psp



In [ ]:
from data.txts import txt_list
from data.csvs import csv_list
from data.jsons import json_list

In [ ]:
print(len(txt_list))
print(len(csv_list))
print(len(json_list))


In [ ]:
from data import txt_dir
from data import json_dir
from data import csv_dir

In [ ]:
print(txt_dir)
print(json_dir)
print(csv_dir)

In [ ]:
from functions_and_classes.paper_processing import process_and_score_paper as psp

In [ ]:
tp = psp('533Cottrell')

In [ ]:
tp.s_w_c['1.4.6']

So, those three imports alone are capable of processing a paper by putting in its file name (with no .txt and no directory), scoring the file and pairing it with the relevant bibliographic information.

In [ ]:
from functions_and_classes.lit_level_operations import lit_scores_from_jsons

In [ ]:
ls = lit_scores_from_jsons()

In [ ]:
ls['lit_s_w_c']['1.1.1']

In [ ]:
from functions_and_classes.lit_level_operations import pd_to_csv, cd_to_csv

In [ ]:
from functions_and_classes.lit_level_operations import lit_to_csv

In [ ]:
lit_to_csv(ls)

In [ ]:
import pandas as pd

In [ ]:
from functions_and_classes.comparisons import s_l_p, a_l_p, s_l_c, a_l_c

In [ ]:
from functions_and_classes.comparisons import updateSLCcomps, updateALCcomps, updateSLPcomps, updateALPcomps

Everything below here is basically just to manually update the comparison spreadsheets in a one time way to include the cite counts for each file. each time the update functions are run in the future they'll automatically take care of this information.

In [ ]:
tSLCcomp = pd.read_csv('data/s_l_c_comp.csv', index_col=0)
tALCcomp = pd.read_csv('data/a_l_c_comp.csv', index_col=0)
tSLPcomp = pd.read_csv('data/s_l_p_comp.csv', index_col=0)
tALPcomp = pd.read_csv('data/a_l_p_comp.csv', index_col=0)

In [ ]:
import json

In [ ]:
strict_cite_count = {}
agg_cite_count = {}
for file in json_list:
    fp = json_dir+file
    jfile = open(fp, 'r')
    jd = json.load(jfile)
    strict_cite_count[jd['name']] = jd['totalStrictCites']
    agg_cite_count[jd['name']] = jd['totalAggressiveCites']
scc_df = pd.DataFrame.from_dict(strict_cite_count, orient='index', columns=['totalStrictCites'])
agg_df = pd.DataFrame.from_dict(agg_cite_count, orient='index', columns=['totalAggressiveCites'])
total_strict_cites = scc_df['totalStrictCites'].sum()
total_agg_cites = agg_df['totalAggressiveCites'].sum()
lit_scc = pd.DataFrame([total_strict_cites], index=['Literature'], columns=['totalStrictCites'])
scc_df = scc_df.append(lit_scc)
lit_agg = pd.DataFrame([total_agg_cites], index=['Literature'], columns=['totalAggressiveCites'])
agg_df = agg_df.append(lit_agg)

In [ ]:
tSLPcomp = pd.merge(tSLPcomp, scc_df, left_index=True, right_index=True)
tSLPcomp.to_csv('data/s_l_p_comp.csv')
tALPcomp = pd.merge(tALPcomp, agg_df, left_index=True, right_index=True)
tALPcomp.to_csv('data/a_l_p_comp.csv')
tSLCcomp = pd.merge(tSLCcomp, scc_df, left_index=True, right_index=True)
tSLCcomp.to_csv('data/s_l_c_comp.csv')
tALCcomp = pd.merge(tALCcomp, agg_df, left_index=True, right_index=True)
tALCcomp.to_csv('data/a_l_c_comp.csv')

In [2]:
from data import a_l_p_comp, a_l_c_comp, s_l_p_comp, s_l_c_comp

In [ ]:
#starting to work towards a function that will return
def sim_list(filename, a_or_s, p_or_c, min_list_len):
    if a_or_s == 'a':
        if p_or_c == 'p':
            df = a_l_p_comp
        elif p_or_c == 'c'
            df = a_l_c_comp
        else:
            print('Your third function input must be "p" or "c")
    elif a_or_s == 's': 
        if p_or_c == 'p':
            df = s_l_p_comp
        elif p_or_c == 'c':
            df = s_l_c_comp
        else:
            print('Your third function input must be "p" or "c")
    else:
        print('Your second function input must be "a" or "s"')

def s_l_c_simlist(filename, min_list_len):
    df = pd.merge(s_l_c_comp[filename], s_l_c_comp['totalStrictCites'], left_index=True, right_index=True)
    df[]
        
    

In [ ]:
t_df = pd.merge(s_l_c_comp['Literature'], s_l_c_comp['totalStrictCites'], left_index=True, right_index=True)

In [ ]:
def n_strict_filter(df_in, num):
    return df_in['totalStrictCites'] > num

In [ ]:
def m_strict_similar(filename, min_cites, min_similar, df_in):
    min_cit = max(0,min_cites)
    min_c = n_strict_filter(df_in, min_cit)
    min_si = min(min_similar,len(df_in[min_c].index)-1)
    sim_score_to_beat = 1.05 * df_in[min_c].sort_values(filename)[min_si:min_si+1][filename].values[0]
    score_to_beat_filter = df_in[min_c][filename] <= sim_score_to_beat

    return df_in[min_c][score_to_beat_filter].sort_values(filename)
    

In [ ]:
def s_l_c_comp_process(file, minimum_comp_cites, minimum_sim_comps):
    df = pd.merge(s_l_c_comp[file], s_l_c_comp['totalStrictCites'], left_index=True, right_index=True)
    return m_strict_similar(file, minimum_comp_cites, minimum_sim_comps, df)[1:]

def s_l_p_comp_process(file, minimum_comp_cites, minimum_sim_comps):
    df = pd.merge(s_l_p_comp[file], s_l_p_comp['totalStrictCites'], left_index=True, right_index=True)
    return m_strict_similar(file, minimum_comp_cites, minimum_sim_comps, df)[1:]

In [ ]:
s_l_p_comp_process('strawson-evident-connexion', 5, 5)

In [ ]:
def n_agg_filter(df_in, num):
    return df_in['totalAggressiveCites'] > num

def m_agg_similar(filename, min_cites, min_similar, df_in):
    min_cit = max(0,min_cites)
    min_c = n_agg_filter(df_in, min_cit)
    min_si = min(min_similar,len(df_in[min_c].index)-1)
    sim_score_to_beat = 1.05 * df_in[min_c].sort_values(filename)[min_si:min_si+1][filename].values[0]
    score_to_beat_filter = df_in[min_c][filename] <= sim_score_to_beat

    return df_in[min_c][score_to_beat_filter].sort_values(filename)

def a_l_c_comp_process(file, minimum_comp_cites, minimum_sim_comps):
    df = pd.merge(a_l_c_comp[file], a_l_c_comp['totalAggressiveCites'], left_index=True, right_index=True)
    return m_agg_similar(file, minimum_comp_cites, minimum_sim_comps, df)[1:]

def a_l_p_comp_process(file, minimum_comp_cites, minimum_sim_comps):
    df = pd.merge(a_l_p_comp[file], a_l_p_comp['totalAggressiveCites'], left_index=True, right_index=True)
    return m_agg_similar(file, minimum_comp_cites, minimum_sim_comps, df)[1:]

In [ ]:
a_l_c_comp_process('Literature',5,5)['Literature'].plot.bar()

In [ ]:
s_l_c_comp_process('Literature', 10, 99)['Literature'].plot.bar()

In [ ]:
a_l_p_comp_process('Literature', 10, 99)['Literature'].plot.bar()

In [ ]:
s_l_p_comp_process('Literature', 10, 99)['Literature'].plot.bar()

OK so far then i have functions that can handle each of the 4 types of comp data across s/a, p/c and take as optional inputs the numbers of cites in the comparison data and the minimum number of comps to return.

i think what i need to do is to make the num of cites and the min num of comps to return optional and specifiable with kws with default arguments of 1 for the min num of cites and 100 for the num num of comps. that way the default will pull the entire body (maybe display in groups of 20 at a time).

then i'll want to convert or actually just add on the paper title as a column the paper title attached to the file name for display purposes. I think it'd be good to add that on at the stage i'm attaching total cite counts. add an author and title column when i pull those from the json files and add them as columns to the intermeidate little df i merge with the comparative scores.

then it would be good to wrap all of this inside a function that takes a/s and c/p as parameters and calls the right function of these four i've developed based on the a/s and c/p parameters passed into the function.

In [ ]:
def article_comps(filename, min_num_cites=1, min_num_comps=100, a_or_s="a", p_or_c="p"):
    if a_or_s == "a":
        if p_or_c == "p":
            return a_l_p_comp_process(filename, minimum_comp_cites=min_num_cites, minimum_sim_comps=min_num_comps)
        elif p_or_c == "c":
            return a_l_c_comp_process(filename, minimum_comp_cites=min_num_cites, minimum_sim_comps=min_num_comps)
        else:
            return print('be sure to choose "p" or "c" for the p_or_c argument')
    elif a_or_s == "s":
        if p_or_c == "p":
            return s_l_p_comp_process(filename, minimum_comp_cites=min_num_cites, minimum_sim_comps=min_num_comps)
        elif p_or_c == "c":
            return s_l_c_comp_process(filename, minimum_comp_cites=min_num_cites, minimum_sim_comps=min_num_comps)
        else:
            return print('be sure to choose "p" or "c" for the p_or_c argument')       
    else: 
        return print('be sure to choose "a" or "s" for the a_or_s argument')

In [ ]:
article_comps('533Cottrell', a_or_s="s", p_or_c="c")

In [1]:
from functions_and_classes.comparisons import article_comps

In [2]:
article_comps('533Cottrell', a_or_s="s", p_or_c="c")

,533Cottrell,totalStrictCites
acpq_1995_0069_0003_0485_0501&pdfname=acpq_1995_0069_0003_0091_0107,67.529471,29
acprof-9780199229505-chapter-6,81.108356,51
1173636,86.036092,102
1173780,88.218218,60
strawson-evident-connexion,88.680060,330
27745095,89.743717,64
Literature,91.534705,2622
project_muse_553401,96.278860,145
project_muse_439820,110.332592,25
Levin2007_Article_BundlingHumeWithKripkenstein,110.440354,4


In [4]:
from data import a_l_p_comp, a_l_c_comp, s_l_p_comp, s_l_c_comp

In [6]:
article_comps('533Cottrell', a_or_s="s", p_or_c="c")

NameError: name 's_l_c_comp' is not defined